In [33]:
import numpy as np

In [34]:
class EWAagent:
    def __init__(self, id, phi=0.1, lam=2, n_options=2, q_init=0.1):
        # properties
        self.id = id
        self.phi = phi
        self.lam = lam
        self.choice_f = lambda x: np.exp(x)/sum(np.exp(x)) # TODO parameter?

        # arrays to hold behavioural history
        self.Q_vals = [tuple(q_init for i  in range(n_options))]
        self.choices = []
        self.correct = []
        self.payoffs = []
        
        self.n_actions = n_options # TODO hmm

    def update_Q_vals(self, values: tuple[float, ...]):
        self.Q_vals.append(values)

    def choose_action(self) -> int:
        choice_probabilities = self.choice_f(self.Q_vals[-1])
        choice = np.random.choice(self.n_actions, 1, p=choice_probabilities).item()
        self.choices.append(choice)
        return choice

    def __repr__(self):
        return f"Agent {self.id}"


ags = [EWAagent(i) for i in range(3)]
ags[0].update_Q_vals((3.3,3.3))
ags[0].Q_vals

[(0.1, 0.1), (3.3, 3.3)]

In [35]:
softmax = lambda x: np.exp(x)/sum(np.exp(x))

softmax((4,2))

array([0.88079708, 0.11920292])

In [36]:
np.random.binomial(1, p=0.2)

0

In [51]:
class Env:
    def __init__(self, 
                 n_agents=1, 
                 n_options=2, 
                 payoff_structure=(0.6, 0.59, 0)
                 ):
        self.n_agents = n_agents
        self.n_options = n_options
        # payoffs
        self.payoff_better, self.payoff_worse, \
            self.payoff_sd = payoff_structure


    def return_rewards(self, choice: int) -> float:
        # TODO
        better = np.random.normal(self.payoff_better, self.payoff_sd)
        worse = np.random.normal(self.payoff_worse, self.payoff_sd)

        return better if (choice==1) else worse
    
    def __repr__(self):
        return f"""MAB env\nAgents: {self.n_agents}\nSize of action space: {self.n_options}
Rewards (high, low, SD): {self.payoff_better, self.payoff_worse, self.payoff_sd}"""

e = Env()
a = EWAagent("A")

e

MAB env
Agents: 1
Size of action space: 2
Rewards (high, low, SD): (0.6, 0.59, 0)

In [6]:
def sim(
        Tmax=100, 
        n_agents=1, 
        n_options=2,
        payoff_better=0.6,
        payoff_worse=0.59,
        payoff_sd=0,
        phi=0.1,
        lam=2):
    
        agents = [EWAagent(i) for i in range(n_agents)]
        actions = [payoff_better, payoff_worse]

        for trial in range(Tmax):
                for agent in agents:
                        # choose action
                        choice = agent.choose_action()
                        
                        
        return agents[0].choices

choices = sim()
np.mean(choices)

0.54

In [3]:
np.full([4,3], 1)

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])